In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time
from hardware.motor_controller import MotorController

class TrafficSignSystem:
    def __init__(self, model_path, camera_index=0):
        """Initialize the traffic sign recognition system."""
        self.model = load_model(model_path)
        self.camera = cv2.VideoCapture(camera_index)
        self.motor_controller = MotorController()
        
        # Define sign classes (modify based on your model)
        self.sign_classes = {
            0: 'speed_limit',
            1: 'stop',
            2: 'turn_right',
            3: 'turn_left',
            # Add more classes as needed
        }
    
    def preprocess_frame(self, frame):
        """Preprocess camera frame for model input."""
        resized = cv2.resize(frame, (32, 32))
        normalized = resized / 255.0
        return np.expand_dims(normalized, axis=0)
    
    def process_sign(self, sign_class, confidence):
        """Process recognized sign and control vehicle."""
        if confidence < 0.85:  # Confidence threshold
            return
        
        action = self.sign_classes.get(sign_class)
        if action == 'stop':
            self.motor_controller.stop()
        elif action == 'turn_left':
            self.motor_controller.turn_left()
        elif action == 'turn_right':
            self.motor_controller.turn_right()
        elif action == 'speed_limit':
            self.motor_controller.move_forward(speed=50)
    
    def run(self):
        """Main loop for traffic sign recognition system."""
        try:
            while True:
                ret, frame = self.camera.read()
                if not ret:
                    print("Failed to capture frame")
                    continue
                
                # Process frame
                preprocessed = self.preprocess_frame(frame)
                prediction = self.model.predict(preprocessed)
                predicted_class = np.argmax(prediction[0])
                confidence = prediction[0][predicted_class]
                
                # Process sign and control vehicle
                self.process_sign(predicted_class, confidence)
                
                # Display result
                cv2.putText(
                    frame,
                    f"Sign: {self.sign_classes.get(predicted_class, 'Unknown')}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2
                )
                cv2.imshow('Traffic Sign Detection', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        
        finally:
            self.cleanup()
    
    def cleanup(self):
        """Clean up resources."""
        self.camera.release()
        cv2.destroyAllWindows()
        self.motor_controller.cleanup()

if __name__ == "__main__":
    system = TrafficSignSystem('traffic_sign_model.h5')
    system.run()

ModuleNotFoundError: No module named 'hardware.motor_controller'